In [1]:
const name = "ItemSimilarity";

In [2]:
using CSV
using DataFrames
using LinearAlgebra
import NBInclude: @nbinclude
using SparseArrays
@nbinclude("../TrainingAlphas/Alpha.ipynb");

In [3]:
anime_to_uid = DataFrame(CSV.File("../../data/processed_data/anime_to_uid.csv"));
anime = DataFrame(
    CSV.File("../../data/processed_data/anime.csv", ntasks = 1; stringtype = String),
)
anime_to_uid = innerjoin(anime_to_uid, anime, on = "animeid" => "anime_id");

In [4]:
# W = read_params("NeuralExplicitItemCFUntuned")["m"][1].W;

In [5]:
W = read_params(read_params("ExplicitItemCF.powerlaw")["S"])["S"];

In [6]:
for i = 1:size(W)[1]
    W[i, i] = Inf
end;

In [7]:
function get_neighborhood(item, W, K; transpose = false)
    if transpose
        weights = W[item, :]
    else
        weights = W[:, item]
    end
    order = sortperm(weights, rev = true)[1:K]
    order[1:K], weights[order[1:K]]
end;

In [8]:
function get_series(df)
    df = copy(df)
    df[!, :uid] = df[:, :anime] .- 1
    sim = innerjoin(df, anime_to_uid, on = "uid")
    sort(sim, :weight, rev = true)
end;

In [12]:
function display_related_series(mal_uid)
    uid = only(anime_to_uid[anime_to_uid.animeid.==mal_uid, :uid])
    items, weights = get_neighborhood(uid + 1, W, 30)
    df = DataFrame(anime = items, weight = weights)
    get_series(df)
end;

In [13]:
ENV["LINES"] = 30;

In [14]:
display_related_series(2921)

Row,anime,weight,uid,animeid,title,main_picture,alternative_titles,start_date,end_date,synopsis,num_list_users,num_scoring_users,nsfw,medium,status,num_episodes,start_season,source,average_episode_duration,studios,related_anime,recommendations,genres,tags,links
,Int64,Float32,Int64,Int64,String,String?,String,Date,Date,String?,Int64,Int64,String,String,String,Int64,String?,String?,Int64,String,String,String,String,String,String
1,12267,Inf,12266,2921,Ashita no Joe 2,"{'medium': 'https://api-cdn.myanimelist.net/images/anime/3/45028.jpg', 'large': 'https://api-cdn.myanimelist.net/images/anime/3/45028l.jpg'}","{'synonyms': ['Rocky Joe 2'], 'en': ""Tomorrow's Joe 2"", 'ja': 'あしたのジョー２'}",1980-10-13,1981-08-31,"Yabuki Joe is left downhearted and hopeless after a certain tragic event. In attempt to put the past behind him, Joe leaves the gym behind and begins wandering. On his travels he comes across the likes of Wolf Kanagushi and Goromaki Gondo, men who unintentionally fan the dying embers inside him, leading him to putting his wanderings to an end. His return home puts Joe back on the path to boxing, but unknown to himself and his trainer, he now suffers deep-set issues holding him back from fighting. In attempt to quell those issues, Carlos Rivera, a world renowned boxer is invited from Venezuela to help Joe recover.",44270,17238,white,tv,finished_airing,47,"{'year': 1980, 'season': 'fall'}",manga,1477,"[{'id': 65, 'name': 'Tokyo Movie Shinsha'}]","[{'anime_id': 2402, 'relation': 'prequel'}, {'anime_id': 2922, 'relation': 'alternative_version'}]","[{'anime_id': 263, 'num_recommendations': 2}, {'anime_id': 2001, 'num_recommendations': 1}, {'anime_id': 22135, 'num_recommendations': 1}, {'anime_id': 1, 'num_recommendations': 1}]","['Action', 'Combat Sports', 'Drama', 'Shounen', 'Slice of Life', 'Sports']","['action', 'asia', 'bar', 'based on a manga', 'boxing', 'combat', 'combat sports', 'coming of age', 'delinquents', 'drama', 'earth', 'foreign', 'hand to hand combat', 'historical', 'japan', 'language barrier', 'mafia', 'male protagonist', 'manga', 'martial arts', 'military', 'ojou-sama', 'philosophy', 'plot continuity', 'present', 'primarily adult cast', 'primarily male cast', 'rehabilitation', 'shounen', 'slice of life', 'sports', 'tragedy', 'urban', 'violence', 'yakuza']","['https://anilist.co/anime/142051', 'https://myanimelist.net/anime/51478']"
2,7712,0.483603,7711,2402,Ashita no Joe,"{'medium': 'https://api-cdn.myanimelist.net/images/anime/5/64671.jpg', 'large': 'https://api-cdn.myanimelist.net/images/anime/5/64671l.jpg'}","{'synonyms': ['Rocky Joe', 'Champion Joe'], 'en': ""Tomorrow's Joe"", 'ja': 'あしたのジョー'}",1970-04-01,1971-09-29,"Joe Yabuki is a troubled youth, whose only solution to problems is throwing punches at them. What he lacks in manners and discipline, he makes up for with his self-taught fighting skills.\n\nOne day, while wandering the slums of Doya, Joe gets into a fight with the local gang. Although greatly outnumbered, he effortlessly defeats them, drawing the attention of Danpei Tange—a former boxing coach turned alcoholic. Seeing his potential, he offers to train Joe into Japan's greatest boxer. At first, Joe dismisses Danpei as a hopeless drunk; but after the trainer saves his life, he agrees to live with him and learn the art of boxing. Unfortunately, Joe's personality makes him an unruly student, and he often falls back to his old ways.\n\nTo survive the harsh world of his new career, Joe needs to trust his mentor and master the techniques taught to him. However, the road to becoming a professional boxer is rife with struggles that will test his mettle to the end.\n\n[Written by MAL Rewrite]",79891,20647,white,tv,finished_airing,79,"{'year': 1970, 'season': 'spring'}",manga,1500,"[{'id': 68, 'name': 'Mushi Production'}]","[{'anime_id': 2920, 'relation': 'alternative_version'}, {'anime_id': 2921, 'relation': 'sequel'}, {'anime_id': 4298, 'relation': 'other'}, {'anime_id': 19973, 'relation': 'prequel'}, {'